In [13]:
# with open('trial-p2-2.txt') as f:
with open('day10.txt') as f:
    lines = f.read()
    map = lines.split("\n")

# print(map)

loop_size = 0
for p in map:
    if p in [ "|", "-", "L", "J", "7", "F", "S" ]:
        loop_size+=1

print(loop_size/2)
# 9520 too high
# 9521 too high

0.0


In [14]:
class Pipe:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.p1 = None
        self.p2 = None
        self.shape = map[x][y]

        #for p2
        self.is_from_loop = False
        self.is_outside = None
    
    def see_next_pipe(self, origin_pipe):
        if origin_pipe==self.p2:
            return self.p1
        return self.p2
            

In [15]:
import pprint


map_pipe = []
def create_map_pipe():
    map_pipe = []
    for m in range(len(map)):
        pipe_line = []
        for p in range(len(map[m])):
            pipe_line.append(Pipe(m, p))
        map_pipe.append(pipe_line)
    return map_pipe

def assign_connected_pipes(pipe):
    # | is a vertical pipe connecting north and south.
    if pipe.shape == "|":
        possible_coords = [ (pipe.x+1, pipe.y), (pipe.x-1, pipe.y) ]
    # - is a horizontal pipe connecting east and west.
    elif pipe.shape == "-":
        possible_coords = [ (pipe.x, pipe.y+1), (pipe.x, pipe.y-1) ]
    # L is a 90-degree bend connecting north and east.
    elif pipe.shape == "L":
        possible_coords = [ (pipe.x-1, pipe.y), (pipe.x, pipe.y+1) ]
    # J is a 90-degree bend connecting north and west.
    elif pipe.shape == "J":
        possible_coords = [ (pipe.x-1, pipe.y), (pipe.x, pipe.y-1) ]
    # 7 is a 90-degree bend connecting south and west.
    elif pipe.shape == "7":
        possible_coords = [ (pipe.x+1, pipe.y), (pipe.x, pipe.y-1) ]
    # F is a 90-degree bend connecting south and east.
    elif pipe.shape == "F":
        possible_coords = [ (pipe.x+1, pipe.y), (pipe.x, pipe.y+1) ]
    else :
        possible_coords = False
    # S is the starting point
    # . is ground; there is no pipe in this tile.
    if possible_coords:
        pipe.p1 = find_pipe_via_coords(possible_coords[0])#
        pipe.p2 = find_pipe_via_coords(possible_coords[1])#map_pipe[possible_coords[1][0]][possible_coords[1][1]]
        if pipe.p1 and pipe.p1.shape =="S":
            pipe.p1.p1 = pipe
        if pipe.p2 and pipe.p2.shape =="S":
            pipe.p2.p1 = pipe

def find_pipe_via_coords(coords):
    if len(map_pipe)>coords[0] and len(map_pipe[coords[0]])>coords[1]:
        return map_pipe[coords[0]][coords[1]]
    return None

map_pipe = create_map_pipe()
for pipe_line in map_pipe:
    for pipe in pipe_line:
        assign_connected_pipes(pipe)

for pipe_line in map_pipe:
    for pipe in pipe_line:
        if pipe.shape =="S":
            start = pipe

current_pipe = start.p1
current_origin = start
counter = 1
while current_pipe.shape != "S":
    next_pipe = current_pipe.see_next_pipe(current_origin)
    current_origin = current_pipe
    current_pipe = next_pipe
    counter+=1

print("cycle : ", counter)
print("farhest", counter/2)





cycle :  13884
farhest 6942.0


In [16]:
# we mark from loop and outside loop elements
current_pipe = start.p1
current_pipe.is_from_loop = True
current_origin = start
current_origin.is_from_loop = True
while current_pipe.shape != "S":
    next_pipe = current_pipe.see_next_pipe(current_origin)
    current_origin = current_pipe
    current_pipe = next_pipe
    current_pipe.is_from_loop = True

def crossing_number(pipe):
    # F7 schould count as 2 
    # F--7 schould count as 2 also
    # F-7 schould count as 2 also
    crossing_number = 0
    for i in range(pipe.y, len(map_pipe[pipe.x])):
        if map_pipe[pipe.x][i].is_from_loop:
            shape = map_pipe[pipe.x][i].shape
            if shape == "S":
                shape = "J"
                
            if shape in "LF":
                buffer = shape
            if shape == "7":
                if buffer == "L":
                    crossing_number+=1
            if shape == "J":
                if buffer == "F":
                    crossing_number+=1
            if shape == "|":
                crossing_number+=1

    return crossing_number%2

outside_counter = 0
inside_counter = 0

map_side = []
for pipe_line in map_pipe :
    map_side_line = ""
    for pipe in pipe_line:
        if pipe.is_from_loop == False:
            cn = crossing_number(pipe)
            if cn%2==0:
                outside_counter+=1
                # map_side_line+="I"
            else :
                inside_counter+=1
                # map_side_line+="O"
            map_side_line+=str(cn)
        else:
            map_side_line+="*"
    map_side.append(map_side_line)

pprint.pprint(map_side)
print(outside_counter)
print(inside_counter)


['00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000**00000000000000000000000000**00000000000000000000000000',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000****0000000000000000000000****00000000000000000000000000',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000*****000000000000000000000****00000000000000